# 📔 READ_ME und TO_DO

#### README

In [1]:
%%writefile '/content/drive/MyDrive/Colab Notebooks/SEO/README.md'
# 🚀 SEO Automation Pipeline mit OpenAI & Retrieval (RAG)

Dieses Projekt bietet eine **komplette End-to-End-Pipeline für die SEO-Optimierung von Websites**, inklusive **Web-Crawling, SEO-Analyse, KI-gestützter Text-Optimierung und Qualitätskontrolle**.

Kern des Projekts sind **automatisierte Abläufe**, die von der **Datengewinnung bis zur SEO-optimierten Textgenerierung** reichen.
Mithilfe von **OpenAI (ChatGPT)** und einer **Retrieval Augmented Generation (RAG)-Architektur** wird sichergestellt, dass die finalen Texte nicht nur **SEO-freundlich**, sondern auch **grammatikalisch korrekt und hochwertig** sind.

## 📚 Inhaltsverzeichnis

- Features
- Projektstruktur
- Ablauf & Module
- Technologien
- Installation
- Nutzung
- Ziele
- Roadmap

## ✅ Features

- 🌐 **Automatisiertes Web Crawling** (inkl. Filter für relevante Inhalte)
- ✍️ **Generierung von SEO-optimierten Texten** mithilfe der OpenAI API
- 🧠 **RAG-gestützte Fehlererkennung & Textkorrektur** mit Vektordatenbank (FAISS)
- 📊 **Analyse der Optimierungsergebnisse** (Statistiken, Ähnlichkeiten, Visualisierungen)
- 📈 **Keyword-Analyse und Keyword-Optimierung**
- 📦 Ausgabe in **HTML und PDF** für Kunden
- 📊 Umfangreiche **Datenvisualisierungen** (Wordclouds, Cosine Similarity, Keyword-Verteilung)




<img src="https://drive.google.com/uc?id=10oR2bcugvN2MClp14ia7gnzMGX5b896t" alt="SEO Heatmap" width="600">




## 🗂️ Projektstruktur

```
SEO-Project/
├── data/                # Prompts, Fehler-Korrektur-Daten, weitere JSON Dateien
├── notebooks/           # Colab/Notebooks zum Starten und Entwickeln
├── output/              # Erzeugte Dateien (HTML, PDF, Bilder)
│   ├── final           # Dokumente für Kunden (HTML, PDF)
│   └── images          # Visualisierungen
├── src/                # Source Code (Python-Klassen und Module)
│   ├── webscraper.py    # Webscrawling und Text-Extraktion
│   ├── llmprocessor.py # Anbindung an OpenAI API, Keyword Extraktion
│   ├── chatbot.py       # Zentrale Chatbot-Klasse zur Kommunikation mit GPT
│   ├── seoanalyzer.py   # Analyse und Auswertung der Texte
│   ├── github.py        # Automatischer Upload ins GitHub Repo
│   ├── utils.py         # Hilfsmodule (z.B. für Prompt-Management)
│   └── embeddingdemo.py# 3D Embedding- und Cosine Similarity Visualisierungen
├── tests/              # pytest der Hauptfunktionalitäten
└── requirements.txt    # Python-Abhängigkeiten
```

## ⚙️ Ablauf & Module

### 1. **Web Crawling**
- **src/webscraper.py**: Holt Inhalte von Webseiten, filtert irrelevante Seiten (z.B. Impressum, AGB).

### 2. **SEO-Optimierung mit OpenAI**
- **src/llmprocessor.py**:
  - Extrahiert Keywords aus den Inhalten.
  - Optimiert die Texte für SEO mit gezielten Prompts.

### 3. **Analyse & Visualisierung**
- **src/seoanalyzer.py**: Verarbeitet und analysiert die Original- und optimierten Texte.

### 4. **GitHub Automation**
- **src/github.py**: Lädt finale Ergebnisse in ein GitHub-Repo hoch.

## 🧰 Technologien

| Technologie                  | Beschreibung                                       |
|-----------------------------|---------------------------------------------------|
| Python                      | Hauptsprache                                       |
| OpenAI API (ChatGPT, GPT-4)  | Generative KI für SEO-Texte                       |
| FAISS                      | Vektorsuche für RAG und Text-Fehler                |
| Pandas, NumPy               | Datenanalyse und Verarbeitung                      |
| Matplotlib, Seaborn         | Visualisierungen                                   |
| Sentence Transformers       | Embedding-Erstellung für Vektordatenbank          |
| BeautifulSoup, Requests     | Webcrawling                                        |
| Google Colab                | Entwicklung und Ausführung                        |

## 🚀 Installation

```bash
pip install -r requirements.txt
python -m spacy download de_core_news_sm
pip install faiss-cpu sentence-transformers openai wordcloud matplotlib seaborn
```

## 💻 Nutzung

```python
scraper = WebsiteScraper(start_url="https://www.example.com")
scraper.scrape_website()

llm_processor = LLMProcessor(prompts_folder, get_filtered_texts, google_ads_keywords)
llm_processor.run_all()

seo_analyzer = SEOAnalyzer(seo_json, original_texts, keywords_final)
seo_analyzer.run_analysis()
```

## 🎯 Ziele

- ✅ Vollständige Automatisierung der SEO-Optimierung
- ✅ RAG für sprachliche Qualitätskontrolle
- ✅ Kundenfertige PDF/HTML-Reports

## 🚧 Roadmap

- [ ] Automatische SEO Scores (z.B. Google Ads API)
- [ ] Automatische Keyword-Erweiterung
- [ ] Mehrsprachigkeit (aktuell Deutsch)
- [ ] WordPress-Integration


Overwriting /content/drive/MyDrive/Colab Notebooks/SEO/README.md


####TODO

In [2]:
%%writefile '/content/drive/MyDrive/Colab Notebooks/SEO/TODO.md'
# To-Do Liste: SEO Automation & KI-Projekt

Diese Liste fasst alle anstehenden Aufgaben im Projekt zusammen

---

## 0. **Aktuelles und dringendes**
- [ ] 18.3.2025 **Version missmatch**: numpy 2.2.3 und spacy 3.5 **side effects on**: dependencies.py, excelimporter.py, Installation.ipynb **ursachen**: spacy version 3.5 verlangt numpy 1.26.4 -> version missmatch
      -> 23.5. class SEOAnlyzer refaktoriert und spacy ersetzt
- [ ] 24.3.25 class **webscraper** meta titel?
- [ ] 28.3.25 lokale SEO keywords liefern manchmal falsche Stadt

---

## 1. **Allgemeine Projektorganisation**
- [ ] **Projektstruktur verbessern**: Ordner übersichtlich gestalten (z.B. `src/`, `data/`, `tests/`, `notebooks/`, dependencies.py).
- [ ] **Dokumentation erweitern**: READ_ME und Wiki (bzw. GitHub Pages) zu jedem Modul anlegen.
- [ ] **Automatisierte Tests** Pytest für Kernfunktionen ausbauen.
- [ ] **Produkt für Kunden finalisieren**
- [ ] **FAISS DB**: automatisierte Erweiterung bei neu gefundenen Fehlern
- [ ] **Template GitHub**: issues
- [ ] Funktionalitäten aus **utils.py** überdenken
- [ ] langfristig Umstieg auf **langchain**
- [ ] textprocessor durch openai **function calling** ersetzen
- [ ] **dependencies** und versionen robuster machen
- [ ] **bug reporting system** einrichten

---

## 2. **Vector-Datenbank (FAISS) & Retrieval**
- [ ] **VectorDB-Klasse finalisieren**:
  - [ ] Kleinere Bugs beheben
  - [ ] Userfreundliche Methoden für neue Einträge
- [ ] **Einrichtung der DB** bei Projektstart (Neubau vs. Laden) vereinheitlichen
- [ ] **Konfigurierbare Ähnlichkeits-Schwelle** (z.B. `threshold=0.6`) besser dokumentieren
- [ ] **Dynamische Filter** für bestimmte Fehlerkategorien (z.B. Stil vs. Grammatik) überlegen
- [ ] **hybrides system mit knowledge tree und RAG** etablieren

---

## 3. **SEO-Optimierungs-Pipeline (LangChain)**
- [ ] **Prompts in JSON-Dateien** verlagern (z.B. `/data/prompts/`) und sauber verlinken
- [ ] **Supervisor-Feedback** integrieren & QA-Schritte definieren

---

## 4. **SEOGrammarChecker & PromptManager**
- [ ] Klassenrefactoring:
  - [ ] **`VectorDB`** vs. **`PromptManager`** vs. **`SEOGrammarChecker`** sauber trennen
  - [ ] Möglichst wenig Code-Duplikate, mehr modulare Testbarkeit
- [ ] **Konfigurationsdatei** (z.B. YAML) für Pfade, wie `FAISS_PATH` & Promptordner
- [ ] **Erweiterbare Prompt-Templates**:
  - [ ] Z.B. `seo_optimization.json`, `grammar_check.json`, `supervisor.json`, etc.

---

## 5. **Abschluss & Integration**
- [ ] **Dokumentation** aller Pipelines & Klassen in der README (oder in separater Doku)
- [ ] **Optionale WordPress-Integration** in der Zukunft (Ideenspeicher)
  - [ ] Upload via REST API
  - [ ] Metadaten (Title, Slug, Tags etc.)


Overwriting /content/drive/MyDrive/Colab Notebooks/SEO/TODO.md


#🌋HYPERPARAMETER



In [34]:
### HYPERPARAMETER ###

# START_URL = 'https://www.ewms-tech.com/'
START_URL = "https://www.rue-zahnspange.de/"
# START_URL = 'https://www.malerarbeiten-koenig.de/'


EXCLUDED_WEBSITES = ["impressum", "datenschutz", "datenschutzerklärung", "agb"]



from google.colab import drive, userdata
drive.mount('/content/drive', force_remount=True)

PROJECT_ROOT = userdata.get("gdrive_seo_root")
PROJECT_ROOT_ESC_STR = PROJECT_ROOT.replace('Colab Notebooks', 'Colab\ Notebooks')

SRC_PATH, DATA_PATH, TEST_PATH, OUTPUT_PATH = PROJECT_ROOT + "/src", PROJECT_ROOT + "/data", PROJECT_ROOT + '/tests', PROJECT_ROOT + '/output'
FAISS_PATH = DATA_PATH + '/faiss_db'
KEYWORD_PATH = DATA_PATH + '/keywords'
PROMPT_PATH = DATA_PATH + '/prompts'


FINAL_IMAGES = {}


Mounted at /content/drive


# 🏁 Install requirements

In [4]:
%run '/content/drive/MyDrive/Colab Notebooks/SEO/notebooks/Installation.ipynb'

In [5]:
# %run '/content/drive/MyDrive/Colab Notebooks/SEO/src/dependencies.py'

# ⛩ push to github

In [33]:
import importlib
import github
importlib.reload(github)
from github import GitHubManager

# Starte den GitHub-Sync
git_manager = GitHubManager(
    userdata.get("github_pat"),
    userdata.get("github_email"),
    userdata.get("github_username"),
    userdata.get("github_repo_seo"),
    PROJECT_ROOT_ESC_STR
)

git_manager.clone_repo()  # Klonen des Repos
git_manager.sync_project()

📥 Klonen des GitHub-Repositories...
📂 Kopiere das Projekt ins Repository...
🔧 Git-Konfiguration wird gesetzt...
➕ Änderungen hinzufügen...
📌 Änderungen committen...
⬆️ Änderungen werden auf GitHub gepusht...
✅ Repository erfolgreich synchronisiert!


# 🕸 crawl

In [48]:
import importlib
import webscraper
importlib.reload(webscraper)
from webscraper import WebsiteScraper

scraper = WebsiteScraper(start_url=START_URL, max_pages=20, excluded_keywords=EXCLUDED_WEBSITES)

original_texts = scraper.get_filtered_texts()

# 📺 google ads seo keywords

In [35]:
import excelimporter
importlib.reload(excelimporter)
from excelimporter import ExcelImporter

importer = ExcelImporter(project_folder=PROJECT_ROOT, header=2)
keyword_df = importer.import_all()

if keyword_df is not None:
    excluded_seo_keywords = ['spange de', 'kfo zentrum essen', 'gerade zahne', 'zahn spange']

    keyword_df = keyword_df[(keyword_df['Avg. monthly searches'] > 10) &
    (~keyword_df['Keyword'].isin(excluded_seo_keywords))
    ].sort_values(by='Avg. monthly searches', ascending=False).reset_index(drop=True).copy()

    google_ads_keywords = list(keyword_df['Keyword'])

    keyword_df.set_index(keyword_df.columns[0], inplace=True)
    searches_df = keyword_df[[col for col in keyword_df.columns if col.startswith('Searches')]]

else:
    print("Error: importer.import_all() returned None. Check your data file and ExcelImporter class.")
    google_ads_keywords = None

✅ Geladen: /content/drive/MyDrive/Colab Notebooks/SEO/data/keywords/Keyword Stats 2025-03-09 at 12_41_38.xlsx


In [37]:
import keywordvisualizer
importlib.reload(keywordvisualizer)
from keywordvisualizer import KeywordVisualizer


visualizer = KeywordVisualizer(
    df=searches_df,
    output_dir=OUTPUT_PATH+'/images',
    image_paths=FINAL_IMAGES
)
visualizer.heatmap()

ℹ️ Logarithmische Skalierung aktiviert.
✅ Heatmap gespeichert als: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/heatmap_2025.png


'/content/drive/MyDrive/Colab Notebooks/SEO/output/images/heatmap_2025.png'

# 🔮webtext analysis + SEO

In [10]:
import llmprocessor
importlib.reload(llmprocessor)
from llmprocessor import LLMProcessor

google_ads_keywords = None if not google_ads_keywords else google_ads_keywords

llm_processor = LLMProcessor(PROMPT_PATH, original_texts, google_ads_keywords=google_ads_keywords)

optimized_texts = llm_processor.run_all()

✅ Google Ads Keywords werden verwendet. LLM Keyword Extraktion wird übersprungen.
📍 Stadt für SEO-Kontext wird via LLM ermittelt...
🚀 Starte SEO-Optimierung mit LLM...

=== SEO-Optimierung für https://www.rue-zahnspange.de/ ===
Analyse

Der derzeitige Text für die Zahnklinik in Essen-Rüttenscheid enthält eine Vielzahl an relevanten Informationen über die angebotenen Behandlungen, eine klare Kommunikation der Praxisschwerpunkte und die Qualifikationen des Teams. Allerdings besteht Verbesserungsbedarf in der Keyword-Optimierung und strukturellen SEO-Qualität. Die vorhandenen SEO-Keywords sind nicht konsequent integriert, was die Sichtbarkeit des Textes in Suchmaschinenergebnissen beeinträchtigen könnte. Zudem könnten einige Abschnitte eine präzisere Ansprache der Zielgruppe erhielten werden, um die Lesbarkeit und das Engagement zu verbessern.

Die Verwendung von Keywords wie "zahnärzte essen rüttenscheid", "retainer kosten" und "kfo essen" könnte diese Texte optimieren und einen spezifis

# 📁 Textprozessor

In [11]:
import json
import textprocessor
importlib.reload(textprocessor)
from textprocessor import TextProcessor

# JSON mit den SEO-Abschnitten extrahieren
json_output = TextProcessor.extract_sections_to_json(list(optimized_texts.keys()), list(optimized_texts.values()))
seo_json = json.loads(json_output)

# texte bereinigen und hinzufügen
seo_json = TextProcessor.add_cleaned_text(seo_json, original_texts)

# Ergebnis anzeigen
print(json.dumps(seo_json, indent=4, ensure_ascii=False))


{
    "https://www.rue-zahnspange.de/": {
        "Analyse": "Der derzeitige Text für die Zahnklinik in Essen-Rüttenscheid enthält eine Vielzahl an relevanten Informationen über die angebotenen Behandlungen, eine klare Kommunikation der Praxisschwerpunkte und die Qualifikationen des Teams. Allerdings besteht Verbesserungsbedarf in der Keyword-Optimierung und strukturellen SEO-Qualität. Die vorhandenen SEO-Keywords sind nicht konsequent integriert, was die Sichtbarkeit des Textes in Suchmaschinenergebnissen beeinträchtigen könnte. Zudem könnten einige Abschnitte eine präzisere Ansprache der Zielgruppe erhielten werden, um die Lesbarkeit und das Engagement zu verbessern. Die Verwendung von Keywords wie zahnärzte essen rüttenscheid, retainer kosten und kfo essen könnte diese Texte optimieren und einen spezifischeren Suchbezug schaffen. Zudem fehlen Meta-Titel und Longtail-Keywords, die zur besseren Auffindbarkeit in Suchmaschinen beitragen könnten. Allgemein könnte die Struktur durch Bull

# 🔎 SEO Analyses + Statistics

In [38]:
# künstliche SEO-Daten
historical_data = {
    "Date": [
        "2023-01-01", "2023-02-01", "2023-03-01",
        "2023-04-01", "2023-05-01", "2023-06-01"
    ],
    "Organic_Sessions": [200, 220, 250, 400, 450, 480],
    "Conversion_Rate": [0.02, 0.021, 0.022, 0.028, 0.03, 0.031],
    "Average_Time_on_Page": [40, 42, 45, 60, 65, 70]
}

In [39]:
import seoanalyzer
importlib.reload(seoanalyzer)
from seoanalyzer import SEOAnalyzer

exclude_list = ["leila", "graf", "koenig", "bjoern", "könig", 'björn', 'adolf', 'schmidt', 'strasse', 'straße', 'tilo', 'remhof']

keywords_final = json.loads(llm_processor.get_keywords()['keywords_final']) if not google_ads_keywords else google_ads_keywords

seo_analyzer = SEOAnalyzer(
    seo_json=seo_json,
    keywords_final=keywords_final,
    output_dir=OUTPUT_PATH+"/images",
    historical_data=historical_data,
    wordcloud_exclude=exclude_list,
    shared_image_dict=FINAL_IMAGES
)

analysis_paths = seo_analyzer.run_analysis()
print("Analysis Plot Pfade:", analysis_paths)

model_paths = seo_analyzer.run_models()
print("Model Plot Pfade:", model_paths)

all_paths = seo_analyzer.get_all_image_paths()
print("Alle Pfade gesammelt:", all_paths)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


🔍 Ähnlichkeitsanalyse gestartet...
📊 Similarities (Original -> Keywords): [0.00812277 0.04118394 0.01946554 0.0964738  0.00936775 0.02300596
 0.02300596 0.11268826 0.04685409 0.01901233 0.0964738  0.02393999]
📊 Similarities (Optimiert -> Keywords): [0.03166214 0.06596838 0.06640671 0.12670931 0.00918358 0.060409
 0.060409   0.12246769 0.04496353 0.04737438 0.12670931 0.0245084 ]
📊 Visualisierung der Similarities:
✅ Plot gespeichert: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/similarity_scores_2025.png
🌍 Wordcloud-Visualisierung:
✅ Plot gespeichert: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/wordclouds_filtered_2025.png
✅ Plot gespeichert: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/wordclouds_raw_2025.png
Analysis Plot Pfade: {'similarities': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/similarity_scores_2025.png', 'wordcloud_filtered': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/wordclouds_filtered_2025.png', 'wordclo

<Figure size 640x480 with 0 Axes>

✅ Plot gespeichert: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/session_forecast_2025.png
✅ Plot gespeichert: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/conversion_forecast_2025.png
Model Plot Pfade: {'seo_trends': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/seo_trends_2025.png', 'session_forecast': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/session_forecast_2025.png', 'conversion_forecast': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/conversion_forecast_2025.png'}
Alle Pfade gesammelt: {'keyword_heatmap': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/heatmap_2025.png', 'similarity_scores': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/similarity_scores_2025.png', 'wordcloud_filtered': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/wordclouds_filtered_2025.png', 'wordcloud_raw': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/wordclouds_raw_2025.png', 'seo_trends': '/conte

# 🛏 embedding demo

In [40]:
import os
import embeddingdemo
importlib.reload(embeddingdemo)
from embeddingdemo import EmbeddingDemo

demo = EmbeddingDemo(output_dir=OUTPUT_PATH+'/images', final_images=FINAL_IMAGES)
demo.run_all_visualizations()

print("Alle Embedding-Bilder:", demo.get_image_paths())
print("Gemeinsame Bilder (shared dict):", FINAL_IMAGES)


📊 3D Keyword Similarity
✅ Plot gespeichert: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/embedding_keyword_3d_2025.png

🔍 Cosine Similarity Steps
✅ Plot gespeichert: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/cosine_similarity_steps_2025.png

📍 Vergleich als Vektoren (3D)
✅ Plot gespeichert: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/cosine_comparison_3d_2025.png

🧠 BERT Embeddings Visualisierung (plotly)
✅ Plot gespeichert: /content/drive/MyDrive/Colab Notebooks/SEO/output/images/bert_embeddings_3d_2025.png / /content/drive/MyDrive/Colab Notebooks/SEO/output/images/bert_embeddings_3d_2025.svg
Alle Embedding-Bilder: {'keyword_heatmap': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/heatmap_2025.png', 'similarity_scores': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/similarity_scores_2025.png', 'wordcloud_filtered': '/content/drive/MyDrive/Colab Notebooks/SEO/output/images/wordclouds_filtered_2025.png', 'wordcloud_raw': '/

# ⛳ json to pdf + docx

In [47]:
import seoreportexporter
importlib.reload(seoreportexporter)
from seoreportexporter import SEOReportExporter
exporter = SEOReportExporter(seo_json, OUTPUT_PATH, DATA_PATH+"/intro_texts/sections_intro.json", FINAL_IMAGES)
exporter.run_all_exports()

✅ HTML exportiert: /content/drive/MyDrive/Colab Notebooks/SEO/output/final/preview.html
✅ PDF mit Playwright erstellt: /content/drive/MyDrive/Colab Notebooks/SEO/output/final/output.pdf
✅ Konvertierung nach DOCX abgeschlossen: /content/drive/MyDrive/Colab Notebooks/SEO/output/final/output.docx


# 📥 Tests

#### 📃 Doku

In [17]:
%%writefile '/content/drive/MyDrive/Colab Notebooks/SEO/tests/DOKU_TESTS.md'
# ✅ Pytest Template: Chatbot Klasse

## 1. Klassen & Methoden die getestet werden sollen

- **Chatbot**
  - `chat()`
  - `chat_with_streaming()`

---

## 2. Beispielhafte Inputs + erwartete Outputs pro Methode

| Methode                      | Beispiel Input                                                           | Erwartete Ausgabe    |
|-----------------------------|-------------------------------------------------------------------------|----------------------|
| `Chatbot.chat()`             | 'Das ist ein Test. Schreibe als Antwort "Test erfolgreich".'           | "Test erfolgreich"   |
| `Chatbot.chat_with_streaming()` | 'Das ist ein Test. Schreibe als Antwort "Test erfolgreich".'         | "Test erfolgreich"   |

---

## 3. Return-Typen der Methoden

| Methode                      | Rückgabe-Typ |
|-----------------------------|--------------|
| `Chatbot.chat()`             | `str`        |
| `Chatbot.chat_with_streaming()` | `str`     |

---

## 4. Externe Services mocken?

| Service         |  Mocken?                           |
|-----------------|-------------------------------------|
| OpenAI API      |  Nein                                |
| FAISS Index     |  Ja (kleine Test-Datenbank für FAISS) |

---

## 5. Ordnerstruktur für Tests

```bash
/project-root/
    /src/
        chatbot.py
    /tests/
        test_chatbot.py
    /logs/
        test_report.log
    ...
```

---


Overwriting /content/drive/MyDrive/Colab Notebooks/SEO/tests/DOKU_TESTS.md


#### 👷 code

In [18]:
import pytest
pytest.main(['-v', TEST_PATH+'/chatbot_test/chatbot_test.py'])

======================================= test session starts ========================================
platform linux -- Python 3.11.11, pytest-8.3.5, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: langsmith-0.3.18, anyio-4.9.0, typeguard-4.4.2
collecting ... collected 2 items

drive/MyDrive/Colab Notebooks/SEO/tests/chatbot_test/chatbot_test.py::test_chat_method PASSED [ 50%]
drive/MyDrive/Colab Notebooks/SEO/tests/chatbot_test/chatbot_test.py::test_chat_with_streaming_method PASSED [100%]

========================================= warnings summary =========================================
../usr/local/lib/python3.11/dist-packages/_pytest/config/__init__.py:1277
  /usr/local/lib/python3.11/dist-packages/_pytest/config/__init__.py:1277: PytestAssertRewriteWarning:
  
  Module already imported so cannot be rewritten: anyio

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
=================================== 2 passed, 1 warning in

<ExitCode.OK: 0>

# 🎨 Error Collection

In [19]:
error_corrections = {
    "Eine Zahnspange kann Kiefergelenksbeschwerden, Kauen- und Sprechprobleme effektiv behandeln.":
    "Eine Zahnspange kann Kiefergelenksbeschwerden, sowie Kau- und Sprechprobleme effektiv behandeln.",
    "Als in den USA geborene Kieferorthopädin bringt Dr. Meier eine multikulturelle Perspektive mit und spricht neben Deutsch auch Englisch, Swahili sowie über Grundkenntnisse in Arabisch und Anfängerkenntnisse in Spanisch.":
    "Als in den USA geborene Kieferorthopädin bringt Dr. Meier eine multikulturelle Perspektive mit und spricht neben Deutsch auch Englisch und Swahili. Dazu hat sie Grundkenntnisse in Arabisch und Anfängerkenntnisse in Spanisch.",
    "Sie hat ihren Master of Science in Geochemie von der Universität Münster, Deutschland, und hat an der Universität Düsseldorf abgeschlossen.":
    "Sie hat ihren Master of Science in Geochemie von der Universität Münster, Deutschland, und hat an der Universität Düsseldorf promoviert.",
    "Ihre Qualifikationen umfassen nicht nur Fachwissen, sondern auch eine besondere Hingabe zu einem ästhetischen Lächeln.":
    "Sie ist hoch qualifiziert und hat eine besondere Hingabe zu einem ästhetischen Lächeln.",
    "behandlungsorientierte Zahnberatung": "patientenorientierte Beratung",
    "ästehthetisches Lächeln": "ästhetisches Lächeln",
    "Nachdem Ihr Behandlungsplan von der Krankenkasse genehmigt wurde": "Nachdem Ihr Behandlungsplan von der Krankenkasse bestätigt wurde",
    "Der aktuelle Text zur Zahnspangenpraxis": "Der aktuelle Text zur kieferorthopädischen Praxis"
}

In [20]:
new_error_corrections = {"Das ist ein neuer Fehler.": "Das ist ein korrigierter Fehler."}

# 🎨 RAG

In [21]:
import os
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from chatbot import Chatbot

# -------------------------
# 1) VectorDB
# -------------------------

test_text = seo_json[list(seo_json.keys())[0]]["SEO"]

class VectorDB:
    """
    Eine Klasse für alles rund um die Vektordatenbank:
    - Aufbauen & Laden (FAISS)
    - Neue Einträge hinzufügen
    - Querying für Context Retrieval
    """

    def __init__(self, db_folder):
        """
        :param db_folder: Pfad zum Datenbank-Ordner
        """
        self.db_folder = db_folder
        self.index_file = os.path.join(db_folder, "faiss_index.bin")
        self.json_file  = os.path.join(db_folder, "faiss_index.json")

        self.index = None
        self.error_dict = {}

        self.model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

    def build_index(self, error_corrections: dict):
        """
        Baut einen neuen FAISS-Index aus den übergebenen Fehler-Korrektur-Paaren.
        """
        print("🔨 Baue neuen FAISS-Index...")
        os.makedirs(self.db_folder, exist_ok=True)

        self.error_dict = error_corrections
        errors = list(self.error_dict.keys())

        # Embeddings
        embeddings = np.array([self.model.encode(e) for e in errors], dtype="float32")

        # FAISS-Index anlegen
        self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(embeddings)

        # Daten auf Festplatte schreiben
        faiss.write_index(self.index, self.index_file)
        with open(self.json_file, "w", encoding="utf-8") as f:
            json.dump(self.error_dict, f, ensure_ascii=False)

        print(f"✅ Neuer Index + JSON in '{self.db_folder}' erstellt.")

    def load_index(self):
        """
        Lädt einen bereits existierenden FAISS-Index und die Fehler-Daten.
        """
        if not (os.path.exists(self.index_file) and os.path.exists(self.json_file)):
            raise FileNotFoundError("❌ Kein FAISS-Index gefunden. Bitte build_index() aufrufen.")

        print("🔎 Lade vorhandenen FAISS-Index...")
        self.index = faiss.read_index(self.index_file)

        with open(self.json_file, "r", encoding="utf-8") as f:
            self.error_dict = json.load(f)

        print("✅ Index & Fehler-Korrekturen geladen.")

    def add_entries(self, new_error_corrections: dict):
        """
        Fügt weitere Fehler-Korrektur-Paare hinzu, ohne alles neu zu bauen.
        """
        if self.index is None:
            # Versuch zu laden, falls vorhanden
            if os.path.exists(self.index_file) and os.path.exists(self.json_file):
                self.load_index()
            else:
                raise FileNotFoundError("❌ Kein Index vorhanden. Bitte erst build_index() nutzen.")

        # Merge in self.error_dict
        for fehler, korrektur in new_error_corrections.items():
            self.error_dict[fehler] = korrektur

        # embeddings nur für die neuen keys
        new_keys = list(new_error_corrections.keys())
        new_embeds = np.array([self.model.encode(k) for k in new_keys], dtype="float32")

        # An Index anhängen
        self.index.add(new_embeds)

        # Speichern
        faiss.write_index(self.index, self.index_file)
        with open(self.json_file, "w", encoding="utf-8") as f:
            json.dump(self.error_dict, f, ensure_ascii=False)

        print(f"✅ {len(new_keys)} neue Einträge hinzugefügt und Index aktualisiert.")

    def query(self, text: str, top_k=3, threshold=0.6):
        """
        Sucht in der DB nach ähnlichen fehlerhaften Formulierungen.

        :param text: Der zu prüfende Satz/Abschnitt
        :param top_k: Anzahl der gesuchten Ähnlichkeiten
        :param threshold: Distanzschwelle
        :return: Liste [(fehler, korrektur), ...]
        """
        if self.index is None:
            self.load_index()

        embed = np.array([self.model.encode(text)], dtype="float32")
        distances, indices = self.index.search(embed, top_k)

        all_errors = list(self.error_dict.keys())

        results = []
        for i in range(top_k):
          idx = indices[0][i]
          # Sicherstellen, dass idx in den Bereich von all_errors passt
          if idx < len(all_errors):
              if distances[0][i] < threshold:
                  fehler_key = all_errors[idx]
                  korrektur = self.error_dict[fehler_key]
                  results.append((fehler_key, korrektur))
        return results


    def retrieve_context(self, seo_text: str) -> str:
        """
        Durchsucht den seo_text Satz für Satz, holt ggf. Korrekturvorschläge
        und baut einen Kontextstring.
        """
        lines = []
        for s in seo_text.split(". "):
            suggestions = self.query(s)
            for old, new in suggestions:
                lines.append(f"- Fehler: {old} ➝ Verbesserung: {new}")

        if lines:
            return "Bekannte Fehler/Korrekturen:\n" + "\n".join(lines)
        else:
            return "Keine bekannten Fehler gefunden."



db = VectorDB(db_folder=FAISS_PATH)
db.build_index(error_corrections)
db.add_entries(new_error_corrections)
db.retrieve_context(test_text)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

🔨 Baue neuen FAISS-Index...
✅ Neuer Index + JSON in '/content/drive/MyDrive/Colab Notebooks/SEO/data/faiss_db' erstellt.
✅ 1 neue Einträge hinzugefügt und Index aktualisiert.


'Bekannte Fehler/Korrekturen:\n- Fehler: ästehthetisches Lächeln ➝ Verbesserung: ästhetisches Lächeln\n- Fehler: ästehthetisches Lächeln ➝ Verbesserung: ästhetisches Lächeln\n- Fehler: behandlungsorientierte Zahnberatung ➝ Verbesserung: patientenorientierte Beratung'

In [22]:
import os
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from chatbot import Chatbot

# -------------------------
# 2) PromptManager
# -------------------------

class PromptManager:
    """
    Lädt Prompts aus dem /data/prompts Ordner und kombiniert sie mit
    dem Context aus der VectorDB, um einen finalen Prompt zu erstellen.
    """

    def __init__(self, prompts_folder="./data/prompts"):
        """
        :param prompts_folder: Ordner, in dem .json (oder .txt) Prompts liegen
        """
        self.prompts_folder = prompts_folder

    def load_prompt(self, filename: str) -> dict:
        """
        Lädt einen JSON-Prompt aus dem Ordner, z.B. 'grammar_prompt.json'.
        """
        path = os.path.join(self.prompts_folder, filename)
        try:
            with open(path, "r", encoding="utf-8") as f:
                return json.load(f)
        except FileNotFoundError:
            print(f"⚠️ Prompt-Datei {path} nicht gefunden!")
            return {}
        except json.JSONDecodeError:
            print(f"⚠️ Ungültiges JSON in {path}")
            return {}

    def build_final_prompt(self, base_prompt_file: str, context: str, user_text: str) -> (str, str):
        """
        Kombiniert:
         - base_prompt_file (System-/User-Prompts)
         - den 'context' aus der VectorDB
         - den 'user_text' (SEO-Text)
        und gibt final (system_prompt, user_prompt) zurück.
        """
        prompt_data = self.load_prompt(base_prompt_file)

        system_prompt = prompt_data.get("system_prompt", "")
        user_prompt   = prompt_data.get("user_prompt", "")

        # Kontext an system_prompt anhängen
        system_prompt_full = system_prompt

        # SEO-Text an user_prompt anhängen
        user_prompt_full = user_prompt.format(context=context,optimized_text=user_text)

        return (system_prompt_full, user_prompt_full)

pm = PromptManager(prompts_folder=PROMPT_PATH)
context = db.retrieve_context(test_text)
final_prompts = pm.build_final_prompt("grammar_check.json", context, test_text)

In [23]:
from chatbot import Chatbot

# -------------------------
# 3) SEOGrammarChecker
# -------------------------

cb = Chatbot(systemprompt=final_prompts[0], userprompt=final_prompts[1])
final_text = cb.chat()
final_text

'Hier ist der überarbeitete Text mit Korrekturen in Grammatik, Rechtschreibung und Stil:\n\n---\n\n**Moderne Zahnspangen für ein gesünderes Lächeln in Essen-Rüttenscheid**\n\nEntdecken Sie die Zahnarztpraxis RÜ Zahnschiene in Essen-Rüttenscheid und unsere vielseitigen kieferorthopädischen Behandlungen für Kinder und Jugendliche, die ein langanhaltendes und gesundes Lächeln ermöglichen.\n\n**Öffnungszeiten:** Montag bis Freitag 9:00 - 18:00 Uhr\n\n**Hier finden Sie uns:** RÜ Zahnschiene in Essen-Rüttenscheid  \n**Anbindungen:** Vor unserer Tür befindet sich die Haltestelle Martinstraße, die erreichbar ist über:  \n- Bahn: 107, 108, U11 \n- Bus: 142, 160, 161 \n\nDirekt gegenüber der Praxis gibt es einen geräumigen Parkplatz.\n\n**Behandlungen für ein perfektes Lächeln**  \nUnser Ziel ist es, Ihnen nicht nur ein ästhetisch ansprechendes Lächeln zu schenken, sondern auch Ihre gesamte Kiefergesundheit zu verbessern. Wir begleiten Sie auf jedem Schritt dieses Weges mit Professionalität und 

# ⛓ Langchain

In [24]:
import subprocess
from langchain_openai import ChatOpenAI
from google.colab import userdata
import os

subprocess.run(["pip", "install", "--upgrade", "pydantic"])

os.environ['OPENAI_API_KEY'] = userdata.get('open_ai_api_key')

llm = ChatOpenAI(temperature=0,
    model = "gpt-4o-mini-2024-07-18",
    openai_api_key=os.environ['OPENAI_API_KEY'],
)

In [25]:
from typing_extensions import assert_type
from utils import load_prompts

prompts = load_prompts(PROMPT_PATH + '/optimize_seo.json')


system_prompt = prompts["system_prompt"]
user_prompt = prompts["user_prompt"]

test_prompt = f"""{prompts["user_prompt"]}"""
#test_prompt = test_prompt.replace('{keywords}', google_ads_keywords.__str__())

user_prompt = user_prompt.replace('{keywords}', google_ads_keywords.__str__())
user_prompt = user_prompt.replace('{original_text}', seo_json[START_URL]['SEO'])


print(test_prompt)

Optimiere den folgenden Text für SEO, indem du die Keywords sinnvoll integrierst:

Keywords: {keywords}

Text:
{original_text}

Achte auf natürliche Sprache, gute Lesbarkeit und Vermeide Keyword-Stuffing.
Gib den optimierten Text zurück.


In [26]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import (AIMessage, HumanMessage, SystemMessage)

def extract_keywords(text):
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="Du bist ein SEO-Experte, spezialisiert auf Keyword-Recherche."),
        HumanMessage(content=f"""
        Analysiere den folgenden Unternehmens-Text und finde die besten SEO-Keywords.
        Berücksichtige lokale Infos, falls vorhanden.

        Text:
        {text}

        Gib mir eine Liste von Keywords.
        """)
    ])
    # format_messages converts the messages to a list of dictionaries
    messages = prompt.format_messages(text=text)
    response = llm(messages)
    return response.content

In [27]:
def optimize_text_for_seo(text, keywords):
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="Du bist ein professioneller SEO-Texter."),
        HumanMessage(content=f"""
        Optimiere den folgenden Text für SEO, indem du diese Keywords sinnvoll integrierst:

        Keywords: {keywords}

        Achte auf natürliche Sprache, gute Lesbarkeit und Vermeidung von Keyword-Stuffing.

        Text:
        {text}

        Gib mir den optimierten Text zurück.
        """)
    ])
    # format_messages converts the messages to a list of dictionaries
    messages = prompt.format_messages(text=text, keywords=keywords)  # Pass keywords here
    response = llm(messages)
    return response.content


In [28]:
def grammar_and_style_check(text):
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="Du bist ein erfahrener Lektor und Sprachexperte."),
        HumanMessage(content=f"""
        Prüfe den folgenden Text auf Grammatik, Rechtschreibung und Stil.
        Mache den Text flüssig, professionell und fehlerfrei.

        Text:
        {text}

        Gib den verbesserten Text zurück.
        """)
    ])
    # format_messages converts the messages to a list of dictionaries
    messages = prompt.format_messages(text=text)  # Format with text
    response = llm(messages)
    return response.content

In [29]:
def supervisor_check(original_text, keywords, optimized_text, final_text):
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="Du bist ein Supervisor, der SEO- und Textqualität überprüft."),
        HumanMessage(content=f"""
        Hier sind die Arbeitsschritte:

        Ursprünglicher Text:
        {original_text}

        Gefundene Keywords:
        {keywords}

        SEO-optimierter Text:
        {optimized_text}

        Finaler Text (nach Lektorat):
        {final_text}

        Beantworte:
        1. Sind alle wichtigen Keywords sinnvoll eingebaut?
        2. Ist der Text professionell und lesbar?
        3. Verbesserungsvorschläge?
        Wenn alles gut ist, antworte: 'Finaler Text akzeptiert.'
        """)
    ])
    # format_messages converts the messages to a list of dictionaries
    messages = prompt.format_messages(
        original_text=original_text,
        keywords=keywords,
        optimized_text=optimized_text,
        final_text=final_text
    )  # Format with all variables
    response = llm(messages)
    return response.content

In [30]:
def seo_pipeline(original_text):
    # Keywords finden
    print("Schritt 1: Keywords finden...")
    keywords = extract_keywords(original_text)
    print("Gefundene Keywords:", keywords)

    # SEO-Optimierung
    print("\nSchritt 2: SEO-Optimierung...")
    optimized_text = optimize_text_for_seo(original_text, keywords)
    print("SEO-optimierter Text:\n", optimized_text)

    # Grammatikprüfung
    print("\nSchritt 3: Grammatikprüfung...")
    final_text = grammar_and_style_check(optimized_text)
    print("Finaler Text nach Lektorat:\n", final_text)

    # Supervisor
    print("\nSupervisor prüft...")
    supervisor_feedback = supervisor_check(original_text, keywords, optimized_text, final_text)
    print("Supervisor Feedback:\n", supervisor_feedback)

    return final_text


In [31]:
if __name__ == "__main__":
    unternehmens_text = seo_json[START_URL]['alt']

    final_output = seo_pipeline(unternehmens_text)
    print("\n--- Finaler SEO-optimierter Text ---\n")
    print(final_output)

Schritt 1: Keywords finden...


<ipython-input-26-9cd5c8d82dab>:20: LangChainDeprecationWarning:

The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.



Gefundene Keywords: Hier ist eine Liste von potenziellen SEO-Keywords, die auf dem Unternehmens-Text basieren. Diese Keywords berücksichtigen sowohl die angebotenen Dienstleistungen als auch lokale Informationen:

### Allgemeine Keywords
1. Kieferorthopädie
2. Zahnspangen
3. Kinderzahnspangen
4. Jugendliche Zahnspangen
5. Unsichtbare Aligner
6. Traditionelle Brackets
7. Lose Zahnspangen
8. Zahnfehlstellungen korrigieren
9. Mundgesundheit
10. Ästhetisches Lächeln

### Lokale Keywords
11. RÜ Zahnspange
12. Kieferorthopädie [Stadt/Region] (z.B. Kieferorthopädie Berlin)
13. Zahnarztpraxis Martinstraße
14. Kieferorthopäde in [Stadt/Region]
15. Zahnspangen in [Stadt/Region]

### Behandlungsbezogene Keywords
16. Behandlungen für Kinder
17. Behandlungen für Jugendliche
18. Kiefergesundheit verbessern
19. Falsche Bissstellung behandeln
20. Schiefe Zähne korrigieren
21. Zahnspange für Erwachsene
22. Retainer zur Stabilisierung
23. Frühzeitige kieferorthopädische Interventionen

### Service- und 